# 📰 News Scraper for BlockBeats

This notebook scrapes news articles from theblockbeats.info with keyword and date filtering.

## Features:
- ✅ Scrape flash news articles
- ✅ Filter by keywords (Chinese & English)
- ✅ Filter by date range (last N days)
- ✅ Export to CSV
- ✅ Automatic date extraction from article text

## How to use:
1. Run all cells in order
2. Configure your settings in the "Configuration" section
3. Run the scraper
4. Download the CSV file

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install requests beautifulsoup4 lxml python-dateutil -q

## 🔧 Step 2: Import Libraries and Define Core Functions

In [ ]:
import requests
import csv
import time
import re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dateutil import parser as date_parser
from typing import List, Optional, Dict
from dataclasses import dataclass
from urllib.parse import urljoin, urlparse

print("✅ Libraries imported successfully!")

## 📊 Step 3: Define Data Models

In [ ]:
@dataclass
class Article:
    """Represents a scraped news article."""
    url: str
    title: str
    publication_date: Optional[datetime]
    author: Optional[str]
    body_text: str
    scraped_at: datetime
    source_website: str
    matched_keywords: Optional[List[str]] = None

print("✅ Data models defined!")

## 🌐 Step 4: HTTP Client Functions

In [ ]:
def fetch_url(url: str, timeout: int = 30, max_retries: int = 3) -> str:
    """Fetch URL with retry logic."""
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; NewsScraperBot/1.0)',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    }
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response.text
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                raise e

print("✅ HTTP client functions defined!")

## 🔍 Step 5: Parser Functions

In [ ]:
def extract_article_urls(html: str, base_url: str) -> List[str]:
    """Extract article URLs from listing page."""
    soup = BeautifulSoup(html, 'lxml')
    article_urls = []
    
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        
        # Only accept /flash/{number} URLs
        if re.match(r'^/flash/\d+$', href):
            absolute_url = urljoin(base_url, href)
            if absolute_url not in article_urls:
                article_urls.append(absolute_url)
    
    return article_urls


def extract_date_from_body(body_text: str) -> Optional[datetime]:
    """Extract date from body text patterns."""
    # Pattern: "BlockBeats 消息，11 月 11 日，"
    pattern = r'BlockBeats\s*消息\s*，\s*(\d{1,2})\s*月\s*(\d{1,2})\s*日'
    match = re.search(pattern, body_text)
    if match:
        month = int(match.group(1))
        day = int(match.group(2))
        year = datetime.now().year
        try:
            return datetime(year, month, day)
        except ValueError:
            pass
    return None


def parse_article(html: str, url: str) -> Article:
    """Parse article from HTML."""
    soup = BeautifulSoup(html, 'lxml')
    
    # Extract title
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else "No Title"
    
    # Extract author
    author = "@BlockBeats"  # Default for this site
    
    # Extract body text
    body_element = soup.select_one('.flash-top, .flash-top-border')
    if body_element:
        # Remove unwanted elements
        for unwanted in body_element.select('script, style, nav, header, footer'):
            unwanted.decompose()
        body_text = body_element.get_text(separator=' ', strip=True)
    else:
        body_text = ""
    
    if not body_text or len(body_text) < 50:
        raise ValueError("Could not extract article body")
    
    # Extract content starting from 'BlockBeats 消息' and remove footer
    # Use simple string find instead of regex for better reliability
    blockbeats_marker = 'BlockBeats 消息'
    blockbeats_pos = body_text.find(blockbeats_marker)
    
    if blockbeats_pos != -1:
        # Start from BlockBeats 消息
        body_text = body_text[blockbeats_pos:]
        
        # Remove footer content - find the earliest footer marker
        footer_markers = [
            'AI 解读',
            '展开',
            '原文链接',
            '举报',
            '纠错/举报',
            '本平台现已全面集成',
            '热门文章'
        ]
        
        earliest_pos = len(body_text)
        for marker in footer_markers:
            pos = body_text.find(marker)
            if pos != -1 and pos < earliest_pos:
                earliest_pos = pos
        
        # Cut at the earliest footer marker
        if earliest_pos < len(body_text):
            body_text = body_text[:earliest_pos]
    
    body_text = body_text.strip()
    
    # Extract date from body text
    publication_date = extract_date_from_body(body_text)
    
    return Article(
        url=url,
        title=title,
        publication_date=publication_date,
        author=author,
        body_text=body_text,
        scraped_at=datetime.now(),
        source_website="www.theblockbeats.info"
    )

print("✅ Parser functions defined!")

## 🎯 Step 6: Filtering Functions

In [ ]:
def should_save_article(article: Article, days_filter: Optional[int], keywords_filter: Optional[List[str]]) -> bool:
    """Check if article passes filters."""
    # Date filter
    if days_filter is not None and article.publication_date:
        cutoff_date = datetime.now() - timedelta(days=days_filter)
        if article.publication_date < cutoff_date:
            return False
    
    # Keyword filter
    if keywords_filter:
        article_text = f"{article.title} {article.body_text}".lower()
        matched = [kw for kw in keywords_filter if kw.lower() in article_text]
        
        if not matched:
            return False
        
        article.matched_keywords = matched
    
    return True

print("✅ Filtering functions defined!")

## 💾 Step 7: Save to CSV Function

In [ ]:
def save_to_csv(articles: List[Article], filename: str):
    """Save articles to CSV file."""
    fieldnames = ['publication_date', 'title', 'body_text', 'url', 'matched_keywords']
    
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader()
        
        for article in articles:
            row = {
                'publication_date': article.publication_date.strftime('%Y-%m-%d') if article.publication_date else '',
                'title': article.title,
                'body_text': article.body_text,
                'url': article.url,
                'matched_keywords': ', '.join(article.matched_keywords) if article.matched_keywords else ''
            }
            writer.writerow(row)

print("✅ CSV save function defined!")

## 🚀 Step 8: Main Scraper Function

In [ ]:
def scrape_news(target_url: str, max_articles: int, keywords: List[str], days: Optional[int], output_file: str, request_delay: float = 2.0):
    """Main scraping function."""
    print(f"\n{'='*70}")
    print("📰 NEWS SCRAPER")
    print(f"{'='*70}")
    print(f"Target URL:      {target_url}")
    print(f"Max Articles:    {max_articles}")
    print(f"Keywords:        {', '.join(keywords)}")
    if days:
        print(f"Date Filter:     Last {days} days")
    print(f"Output File:     {output_file}")
    print(f"{'='*70}\n")
    
    start_time = time.time()
    articles_scraped = []
    articles_failed = 0
    errors = []
    
    try:
        # Fetch listing page
        print("📡 Fetching article listing page...")
        html = fetch_url(target_url)
        
        # Extract article URLs
        print("🔍 Extracting article URLs...")
        article_urls = extract_article_urls(html, target_url)
        print(f"✅ Found {len(article_urls)} article URLs")
        
        # Limit to max_articles
        article_urls = article_urls[:max_articles]
        print(f"📝 Processing {len(article_urls)} articles\n")
        
        # Process each article
        for index, article_url in enumerate(article_urls, 1):
            try:
                print(f"[{index}/{len(article_urls)}] Processing: {article_url}")
                
                # Fetch article
                article_html = fetch_url(article_url)
                
                # Parse article
                article = parse_article(article_html, article_url)
                
                # Check filters
                if should_save_article(article, days, keywords):
                    articles_scraped.append(article)
                    print(f"   ✅ Scraped: {article.title[:60]}...")
                else:
                    print(f"   ⏭️  Filtered out")
                
                # Rate limiting
                if index < len(article_urls):
                    time.sleep(request_delay)
                    
            except Exception as e:
                articles_failed += 1
                error_msg = f"Failed to scrape {article_url}: {str(e)}"
                errors.append(error_msg)
                print(f"   ❌ Error: {str(e)}")
        
        # Save to CSV
        if articles_scraped:
            print(f"\n💾 Saving {len(articles_scraped)} articles to {output_file}...")
            save_to_csv(articles_scraped, output_file)
        
        # Summary
        duration = time.time() - start_time
        print(f"\n{'='*70}")
        print("📊 SCRAPING SUMMARY")
        print(f"{'='*70}")
        print(f"Articles Found:    {len(article_urls)}")
        print(f"Articles Scraped:  {len(articles_scraped)}")
        print(f"Articles Failed:   {articles_failed}")
        print(f"Duration:          {duration:.2f} seconds")
        print(f"{'='*70}\n")
        
        if articles_scraped:
            print(f"✅ Successfully scraped {len(articles_scraped)} article(s)!")
            print(f"📄 Results saved to: {output_file}")
        else:
            print("⚠️  No articles matched your filters")
            
    except Exception as e:
        print(f"\n❌ Fatal error: {e}")
        raise

print("✅ Main scraper function defined!")

## ⚙️ Step 9: Configuration

**Customize these settings:**

In [ ]:
# ========== CONFIGURATION ==========

# Target website
TARGET_URL = "https://www.theblockbeats.info/newsflash"

# Maximum number of articles to scrape
MAX_ARTICLES = 20

# Keywords to filter (articles must contain at least one)
# You can use Chinese or English keywords
KEYWORDS = ["BTC", "ETH", "Bitcoin", "Ethereum", "Uniswap", "黑客", "监管"]

# Date filter: only scrape articles from last N days (set to None to disable)
DAYS_FILTER = 7  # Last 7 days

# Output filename with timestamp
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
OUTPUT_FILE = f"crypto_news_{timestamp}.csv"

# Delay between requests (seconds) - be respectful to the server!
REQUEST_DELAY = 2.0

# ===================================

print("✅ Configuration set!")
print(f"\nYour settings:")
print(f"  - Max articles: {MAX_ARTICLES}")
print(f"  - Keywords: {', '.join(KEYWORDS)}")
print(f"  - Date filter: Last {DAYS_FILTER} days" if DAYS_FILTER else "  - Date filter: Disabled")
print(f"  - Output file: {OUTPUT_FILE}")

## 🎬 Step 10: Run the Scraper!

**Click the play button to start scraping:**

In [ ]:
# Run the scraper
scrape_news(
    target_url=TARGET_URL,
    max_articles=MAX_ARTICLES,
    keywords=KEYWORDS,
    days=DAYS_FILTER,
    output_file=OUTPUT_FILE,
    request_delay=REQUEST_DELAY
)

## 📥 Step 11: Preview and Download Results

In [ ]:
# Preview the results
import pandas as pd

try:
    df = pd.read_csv(OUTPUT_FILE)
    print(f"\n📊 Preview of {OUTPUT_FILE}:")
    print(f"\nTotal articles: {len(df)}\n")
    print(df[['title', 'publication_date', 'matched_keywords']].head(10))
    
    print(f"\n💡 To download the file:")
    print(f"   1. Click the folder icon on the left sidebar")
    print(f"   2. Find '{OUTPUT_FILE}'")
    print(f"   3. Right-click and select 'Download'")
    
except FileNotFoundError:
    print("❌ No results file found. Please run the scraper first!")

## 📝 Notes

- **Rate Limiting**: The scraper waits 2 seconds between requests by default. Don't set this too low!
- **Keywords**: Articles must contain at least ONE of your keywords
- **Date Format**: Dates are extracted from article text and formatted as YYYY-MM-DD
- **URL Format**: Only scrapes URLs in format `/flash/{number}`

## 🔄 To Run Again

1. Modify the configuration in Step 9
2. Re-run Step 10
3. Preview results in Step 11